<a href="https://colab.research.google.com/github/jonasmotasimoes/digitaliza_ponto/blob/main/Digitalizador_Folha_de_Ponto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai pillow openpyxl


In [ ]:
#google-generativeai`: Para interagir com a API do Google AI Studio (anteriormente conhecido como PaLM API).
#Pillow`: (PIL) Para manipulação de imagens (se você estiver lidando com folhas de ponto em formato de imagem).
#openpyxl`: Para criar e manipular arquivos Excel.

In [1]:
from google.colab import userdata
import google.generativeai as genai

# Obtenha a API Key do segredo do Colab
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure a API Key
genai.configure(api_key=GOOGLE_API_KEY)

# Usando o modelo Gemini flash
model_name = "gemini-2.0-flash"
model_gemini = genai.GenerativeModel(model_name) # Crie uma instância de modelo

# Agora você pode usar model_gemini

prompt = "Qual é a capital da França?"

try:
    response = model_gemini.generate_content(prompt)  # Usando Gemini Flash
    print(response.text)

    # response_bison = model_bison.generate_content(prompt) # Usando Text Bison (se você definiu)
    # print(response_bison.text)

except Exception as e:
    print(f"Erro ao chamar a API: {e}")

A capital da França é Paris.



In [2]:
from google.colab import files

def upload_image():
  """Abre uma caixa de diálogo para upload de imagem e retorna o nome do arquivo."""
  uploaded = files.upload()
  for filename in uploaded.keys():
    print(f'Arquivo "{filename}" carregado com sucesso.')
    return filename  # Retorna o nome do primeiro arquivo carregado
  return None # Retorna None se nenhum arquivo foi carregado

# Exemplo de uso:
nome_arquivo = upload_image()

if nome_arquivo:
  print(f"Nome do arquivo carregado: {nome_arquivo}")
else:
  print("Nenhum arquivo foi carregado.")

Saving IMG-20240630-WA0066.jpg to IMG-20240630-WA0066.jpg
Arquivo "IMG-20240630-WA0066.jpg" carregado com sucesso.
Nome do arquivo carregado: IMG-20240630-WA0066.jpg


In [3]:
import google.generativeai as genai
from google.colab import userdata
from PIL import Image

# Obtenha a API Key do segredo do Colab
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def ler_folha_de_ponto(nome_arquivo):
    """
    Envia a imagem da folha de ponto para o Gemini API e extrai os dados.
    (Assumindo que o Gemini API suporte o envio direto de imagens)
    """

    model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17') # Use um modelo que suporte visão (se disponível)

    try:
        # Abre a imagem
        img = Image.open(nome_arquivo)

        prompt_texto = """
Analise a imagem fornecida de uma lista de presença/folha de ponto manuscrita.
Sua tarefa é extrair as informações de cada linha da tabela principal.
Para cada linha preenchida na tabela, extraia os seguintes campos:
- "Dia do Mês"
- "Nº do Livro"
- "Nº da Adesão"
- "Código da Função"
- "Nome" (o nome da pessoa)
- "Entrada" (o primeiro horário registrado para a pessoa, no formato HH:MM)
- "Saída" (o segundo horário registrado para a pessoa, no formato HH:MM)
- "Assinatura"

 Regras importantes para a extração:
 1. Formato da Saída: Apresente os dados extraídos como uma lista de objetos JSON, onde cada objeto representa uma linha da tabela.
 2. Campo "Assinatura": Se houver qualquer marca, escrita ou assinatura na coluna "Assinatura" para uma determinada linha, o valor para este campo deve ser "Sim". Se a célula da assinatura estiver visivelmente vazia (sem marcas além da linha da grade), o valor deve ser "Não".
 3. Campos Vazios ou Ilegíveis: Se um campo específico (exceto assinatura) estiver vazio, não preenchido, ou completamente ilegível na imagem para uma linha, use null ou uma string vazia "" para o valor desse campo no JSON. Para "Nº da Adesão" e "Saída", se estiverem vazios, use null.
 4. Horários: Extraia os horários no formato HH:MM. Se houver apenas um horário (Entrada) e a Saída estiver em branco, registre a Saída como null ou "".
 5. Foco: Concentre-se apenas nas linhas que contêm nomes de pessoas. Ignore linhas de cabeçalho ou linhas completamente em branco na tabela.
 6. Consistência: Tente ser o mais consistente possível na interpretação da caligrafia.
 7. Precisão do Nome: Tenha cuidado extra ao extrair o campo "Nome". Caligrafia pode variar muito, então tente identificar padrões de letras e palavras completas em vez de apenas caracteres isolados. Se a qualidade da imagem para um nome específico for baixa, considere usar uma string como "Nome Ilegível" ou similar, em vez de uma猜测incorreta.
        """

        # ENVIANDO A IMAGEM DIRETAMENTE (SE A API SUPORTAR):
        response = model.generate_content([prompt_texto, img])  # Envia o prompt e a imagem

        return response.text  # Retorna a resposta (em formato JSON)

    except Exception as e:
        print(f"Erro ao processar a imagem ou chamar a API: {e}")
        return None

In [4]:
# Chame a função para ler a folha de ponto
nome_arquivo_timesheet = nome_arquivo # Substitua pelo nome do arquivo que você carregou
dados_extraidos_texto = ler_folha_de_ponto(nome_arquivo_timesheet)

if dados_extraidos_texto:
    print("Dados extraídos com sucesso (formato de texto):")
    # Mostra somente a tabela (o texto retornado pela API)
    print(dados_extraidos_texto)

else:
    print("Não foi possível extrair os dados da folha de ponto.")

Dados extraídos com sucesso (formato de texto):
```json
[
  {
    "Dia do Mês": "17",
    "Nº do Livro": "31",
    "Nº da Adesão": "12",
    "Código da Função": "222",
    "Nome": "Lourivani de Camargo Silva",
    "Entrada": "07:50",
    "Saída": "16:00",
    "Assinatura": "Sim"
  },
  {
    "Dia do Mês": "20",
    "Nº do Livro": "31",
    "Nº da Adesão": "80",
    "Código da Função": "222",
    "Nome": "Sandres Meire Ferreira Oliveira",
    "Entrada": "06:50",
    "Saída": "11:00",
    "Assinatura": "Sim"
  },
  {
    "Dia do Mês": "20",
    "Nº do Livro": "31",
    "Nº da Adesão": "7",
    "Código da Função": "222",
    "Nome": "Elizabeth MS. Silve",
    "Entrada": "06:57",
    "Saída": "09:30",
    "Assinatura": "Sim"
  },
  {
    "Dia do Mês": "20",
    "Nº do Livro": "31",
    "Nº da Adesão": "121",
    "Código da Função": "222",
    "Nome": "Rolaine Natali de matos Traiatt",
    "Entrada": "07:00",
    "Saída": "10:10",
    "Assinatura": "Sim"
  },
  {
    "Dia do Mês": "20",
   

In [ ]:
import pandas as pd
import json
from IPython.display import display, clear_output


# Certifique-se de que 'dados_extraidos_texto' contém a string extraída da célula anterior.

# Tratamento de erro para garantir que 'dados_extraidos_texto' exista e seja uma string.
if 'dados_extraidos_texto' in locals() and isinstance(dados_extraidos_texto, str):

    # --- INÍCIO: Limpeza da string antes de tentar decodificar como JSON ---
    string_limpa = dados_extraidos_texto.strip() # Remove espaços em branco no início e fim

    # Remove o prefixo '
    if string_limpa.startswith('```json'):
        string_limpa = string_limpa[len('```json'):]
    if string_limpa.endswith('```'):
        string_limpa = string_limpa[:-len('```')]
    # --- FIM: Limpeza da string ---

    try:
        # Tenta decodificar a string JSON limpa em uma lista de dicionários
        dados_json = json.loads(string_limpa)

        # Cria um DataFrame do pandas a partir dos dados JSON
        df_folha_ponto = pd.DataFrame(dados_json)

        # Exibe o DataFrame
        print("DataFrame criado com sucesso:")
        display(df_folha_ponto)

        # --- INÍCIO: Salvar o DataFrame em um arquivo Excel ---
        nome_arquivo_excel = "folha_de_ponto_extraida.xlsx"

        # Use ExcelWriter para garantir que o arquivo seja criado corretamente
        with pd.ExcelWriter(nome_arquivo_excel, engine='openpyxl') as writer:
            df_folha_ponto.to_excel(writer, index=False, sheet_name='Dados Extraídos')

        print(f"\nDataFrame salvo em '{nome_arquivo_excel}' com sucesso.")
        # --- FIM: Salvar o DataFrame em um arquivo Excel ---

        # --- INÍCIO: Fazer o download do arquivo Excel ---
        print(f"Preparando download de '{nome_arquivo_excel}'...")
        files.download(nome_arquivo_excel)
        print("Download iniciado (verifique a barra de downloads do seu navegador).")
        # --- FIM: Fazer o download do arquivo Excel ---


    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON. Verifique se a string está no formato JSON correto: {e}")
        print("String limpa recebida:")
        print(string_limpa) # Imprime a string para depuração
    except Exception as e:
        print(f"Ocorreu um erro ao criar ou exibir o DataFrame: {e}")
else:
    print("'dados_extraidos_texto' não existe ou não é uma string. Execute a célula anterior primeiro.")

DataFrame criado com sucesso:


,Dia do Mês,Nº do Livro,Nº da Adesão,Código da Função,Nome,Entrada,Saída,Assinatura
0,07,31,45,249,Adriana Balman Ribeiro,17:36,20:10,Sim
1,07,31,46,249,Viviane C. A. Santos,17:40,20:18,Sim
2,07,31,47,249,Anibal Ap. dos Santos de Martini,17:46,20:20,Sim
3,16,31,110,249,Priscila Pires Campos,17:40,20:00,Sim
4,16,1,43,249,Silvana M.S. de Freitas,17:55,20:00,Sim
5,16,31,107,249,Murilla de Moura,18:00,20:08,Sim
6,16,31,126,249,Viviani Vasconcelos,18:01,20:00,Sim
7,16,31,137,249,Patricia Sacrato Pala Silva,18:02,20:00,Sim
8,23,31,133,249,Gabrielle Gomes Scarasatte,17:28,20:10,Sim
9,23,31,108,249,Érica C. D. Traglia Leoni,17:35,20:11,Sim



DataFrame salvo em 'folha_de_ponto_extraida.xlsx' com sucesso.
Preparando download de 'folha_de_ponto_extraida.xlsx'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download iniciado (verifique a barra de downloads do seu navegador).
